<a href="https://colab.research.google.com/github/spatank/Curiosity/blob/master/growing_Wiki.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Curiosity_IGT/Wiki')

Mounted at /content/drive


In [2]:
!ls # run !ls to verify location

growing_Wiki.ipynb  wiki_data_processed  wiki_data_raw


In [3]:
import glob
import json
import pandas as pd
# !pip install wikipedia2vec
# from wikipedia2vec import Wikipedia2Vec
# import numpy as np
# from scipy.spatial.distance import cosine
from scipy.io import savemat
import networkx as nx

In [4]:
all_json = glob.glob('wiki_data_raw/*.json')

In [5]:
filtered_json = all_json[-5:]
filtered_json

['wiki_data_raw/physics.json',
 'wiki_data_raw/psychology.json',
 'wiki_data_raw/robotics.json',
 'wiki_data_raw/sociology.json',
 'wiki_data_raw/software engineering.json']

In [6]:
for filename in filtered_json:
  topic = filename.split('/')[1].split('.')[0].split()
  topic = '_'.join(topic)
  !mkdir $topic
  print('Starting ' + topic)
  # load data for current topic
  with open(filename) as f:
    data = json.load(f)
  # find the set of all years where there's been activity in the topic
  years = []
  for node in data['nodes']:
    years.append(node['year'])
  years = sorted(set(years)) # sorted ascending
  # each year with activity gets a network 
  for year_ID, year in enumerate(years):
    # initialize empty graph
    G = nx.DiGraph()
    edge_info = []
    for link in data['links']:
      source_ID = link['source']
      source_node_year = data['nodes'][source_ID]['year']
      target_ID = link['target']
      target_node_year = data['nodes'][target_ID]['year']
      # if both concepts exist by the year under consideration, add edge
      if source_node_year <= year and target_node_year <= year:
        from_node = data['nodes'][source_ID]['id']
        to_node = data['nodes'][target_ID]['id']
        edge_weight = link['weight']
        G.add_edge(source_ID, target_ID, weight = edge_weight)
        edge_info_dict = {'from': from_node, 'to': to_node, 'weight': edge_weight}
        edge_info.append(edge_info_dict)
    try:
      adj_G = nx.linalg.graphmatrix.adjacency_matrix(G, weight = 'weight')
    except:
      adj_G = []
    savename = topic + '/' + topic + '_' + str(year_ID) + '.mat'
    mdic = {'year': year, 'adj': adj_G, 'edge_info': edge_info}
    savemat(savename, mdic)
  print('Ending ' + topic)

Starting physics
Ending physics
Starting psychology
Ending psychology
Starting robotics
Ending robotics
Starting sociology
Ending sociology
Starting software_engineering
Ending software_engineering
